# Langchain Agent

使用 langchain 可以很方便地构造一个 Agent。不过在创建 Agent 之前，首先得知道 Agent 使用什么 Tools。

这个测试之中，Agent 将使用一个本地 RAG 和 一个网络搜索服务

## 0. LLM 配置

In [1]:
from langchain_openai import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader

# LLM
llm = ChatOpenAI(
    openai_api_base="http://localhost:1234/v1",
    openai_api_key="not-needed"
)


# 配置Embedding模型
modelPath = "BAAI/bge-base-en-v1.5"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
# model_kwargs = {'device':'cpu'}
model_kwargs = {'device':'cuda'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': True}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)


# 向量化数据
# 1. 加载数据
loader = WebBaseLoader("https://python.langchain.com/docs/get_started/introduction")
docs = loader.load()

# 2. 向量化处理
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

retriever = vector.as_retriever()

## 1. 定义 Agent 工具

In [2]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langchain_search",
    "Search for information about Langchain. For any questions about Langchain, you must use this tool!",
)

In [3]:
import os

os.environ['OPENAI_API_KEY'] = 'YOUR_API_KEY'

定义 Agent 工具列表

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()
tools = [retriever_tool, search]

在 langchain 中，[langchainHub](https://smith.langchain.com/hub) 是官方分享prompt的社区。可以通过 langchainhub 库进行拉去，方便引入到项目之中

In [5]:
%pip install langchainhub

Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# 拉去社区prompt
prompt = hub.pull("hwchase17/openai-functions-agent")

# 定义agent
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

#"""Whether or not run in verbose mode. In verbose mode, some intermediate logs
#    will be printed to the console. Defaults to the global `verbose` value,
#    accessible via `langchain.globals.get_verbose()`."""

通过agent解决问题

In [7]:
agent_executor.invoke({"input": "how can langchain help with building LLM Application?"})



> Entering new AgentExecutor chain...
LangChain is an AI language model designed to assist with various text-based tasks. While it may not be able to directly build an LLM (Master of Laws) application for you, it can certainly provide valuable support in several ways:

1. Drafting statements of purpose: LangChain can help you craft a compelling statement of purpose by suggesting phrases, structures, and arguments based on your input. It can also help ensure that the language is clear, concise, and professional.
2. Editing personal statements: LangChain can review your draft statements of purpose and suggest improvements in grammar, style, and coherence. It can also provide feedback on content to help you strengthen your application.
3. Preparing for interviews: LangChain can help you prepare for LLM interviews by providing sample questions and suggested responses based on common topics. It can also help you practice answering questions effectively and confidently.
4. Researching law 

{'input': 'how can langchain help with building LLM Application?',
 'output': 'LangChain is an AI language model designed to assist with various text-based tasks. While it may not be able to directly build an LLM (Master of Laws) application for you, it can certainly provide valuable support in several ways:\n\n1. Drafting statements of purpose: LangChain can help you craft a compelling statement of purpose by suggesting phrases, structures, and arguments based on your input. It can also help ensure that the language is clear, concise, and professional.\n2. Editing personal statements: LangChain can review your draft statements of purpose and suggest improvements in grammar, style, and coherence. It can also provide feedback on content to help you strengthen your application.\n3. Preparing for interviews: LangChain can help you prepare for LLM interviews by providing sample questions and suggested responses based on common topics. It can also help you practice answering questions effec